# Sanity checks

In [1]:
import os

os.chdir(os.path.abspath(os.path.join(os.getcwd(), "../..")))
print(os.getcwd())

import pandas as pd

/nas-ctm01/homes/mjvieira/classify-protect-illegal-porn-xai


## Frame extraction

### Pornography-800

In [3]:
paths = [
    "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornEasy",
    "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornDifficulty",
    "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vPorn",
]

#### even-20

In [31]:
# check if data file is available
data_loc_800_even_20 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data-processed/even-20"
os.path.isfile(f"{data_loc_800_even_20}/data.csv")

True

In [32]:
df_800_even_20 = pd.read_csv(f"{data_loc_800_even_20}/data.csv")
df_800_even_20["video"] = [frame_name.split("#")[0] for frame_name in df_800_even_20["frame"]]
df_800_even_20

,frame,label,video
0,vNonPorn152#0.jpg,0,vNonPorn152
1,vNonPorn152#1.jpg,0,vNonPorn152
2,vNonPorn152#2.jpg,0,vNonPorn152
3,vNonPorn152#3.jpg,0,vNonPorn152
4,vNonPorn152#4.jpg,0,vNonPorn152
...,...,...,...
15955,vPorn346#15.jpg,1,vPorn346
15956,vPorn346#16.jpg,1,vPorn346
15957,vPorn346#17.jpg,1,vPorn346
15958,vPorn346#18.jpg,1,vPorn346


In [33]:
agg = {"video": "first", "label": "first", "frame": "size"}
df_800_even_20_videos = df_800_even_20[["video", "frame", "label"]]
df_800_even_20_videos = df_800_even_20_videos.groupby("video").aggregate(agg).reset_index(drop=True)
df_800_even_20_videos = df_800_even_20_videos.rename(columns={"frame": "frame_count"})
df_800_even_20_videos

,video,label,frame_count
0,vNonPorn1,0,20
1,vNonPorn10,0,20
2,vNonPorn100,0,20
3,vNonPorn1006,0,20
4,vNonPorn1007,0,20
...,...,...,...
793,vPorn94,1,20
794,vPorn95,1,20
795,vPorn96,1,20
796,vPorn97,1,20


In [34]:
for path in paths:
    videos_in_path = 0
    for v in os.listdir(path):
        if v.startswith("."):
            continue
        
        videos_in_path += 1
        v_name = os.path.splitext(v)[0]
        
        if len(df_800_even_20_videos[df_800_even_20_videos["video"] == v_name]) == 0:
            print(f"{v_name} not in dataset (it belongs to {path})")
        else:
            df_800_even_20_videos.loc[df_800_even_20_videos["video"] == v_name, "folder"] = os.path.basename(path)
    
    print(f"videos in {path}: {videos_in_path}")

videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornEasy: 200
videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornDifficulty: 198
videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vPorn: 400


In [35]:
df_800_even_20_videos

,video,label,frame_count,folder
0,vNonPorn1,0,20,vNonPornEasy
1,vNonPorn10,0,20,vNonPornDifficulty
2,vNonPorn100,0,20,vNonPornEasy
3,vNonPorn1006,0,20,vNonPornDifficulty
4,vNonPorn1007,0,20,vNonPornDifficulty
...,...,...,...,...
793,vPorn94,1,20,vPorn
794,vPorn95,1,20,vPorn
795,vPorn96,1,20,vPorn
796,vPorn97,1,20,vPorn


In [36]:
# videos from which it was not possible to extract 20 frames
df_800_even_20_videos[df_800_even_20_videos["frame_count"] < 20]

,video,label,frame_count,folder


In [37]:
# NOTE: I accidentally removed one video from its original location

print("number of videos:", len(df_800_even_20_videos))
print("number of non-porn frames:", len(df_800_even_20[df_800_even_20["label"] == 0]))
print("number of porn frames:", len(df_800_even_20[df_800_even_20["label"] == 1]))

number of videos: 798
number of non-porn frames: 7960
number of porn frames: 8000


#### middle-20

In [23]:
# check if data file is available
data_loc_800_middle_20 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data-processed/middle-20"
os.path.isfile(f"{data_loc_800_middle_20}/data.csv")

True

In [24]:
df_800_middle_20 = pd.read_csv(f"{data_loc_800_middle_20}/data.csv")
df_800_middle_20["video"] = [frame_name.split("#")[0] for frame_name in df_800_middle_20["frame"]]
df_800_middle_20

,frame,label,video
0,vNonPorn152#0.jpg,0,vNonPorn152
1,vNonPorn152#1.jpg,0,vNonPorn152
2,vNonPorn152#2.jpg,0,vNonPorn152
3,vNonPorn152#3.jpg,0,vNonPorn152
4,vNonPorn152#4.jpg,0,vNonPorn152
...,...,...,...
15955,vPorn346#15.jpg,1,vPorn346
15956,vPorn346#16.jpg,1,vPorn346
15957,vPorn346#17.jpg,1,vPorn346
15958,vPorn346#18.jpg,1,vPorn346


In [25]:
agg = {"video": "first", "label": "first", "frame": "size"}
df_800_middle_20_videos = df_800_middle_20[["video", "frame", "label"]]
df_800_middle_20_videos = df_800_middle_20_videos.groupby("video").aggregate(agg).reset_index(drop=True)
df_800_middle_20_videos = df_800_middle_20_videos.rename(columns={"frame": "frame_count"})
df_800_middle_20_videos

,video,label,frame_count
0,vNonPorn1,0,20
1,vNonPorn10,0,20
2,vNonPorn100,0,20
3,vNonPorn1006,0,20
4,vNonPorn1007,0,20
...,...,...,...
793,vPorn94,1,20
794,vPorn95,1,20
795,vPorn96,1,20
796,vPorn97,1,20


In [26]:
for path in paths:
    videos_in_path = 0
    for v in os.listdir(path):
        if v.startswith("."):
            continue
        
        videos_in_path += 1
        v_name = os.path.splitext(v)[0]
        
        if len(df_800_middle_20_videos[df_800_middle_20_videos["video"] == v_name]) == 0:
            print(f"{v_name} not in dataset (it belongs to {path})")
        else:
            df_800_middle_20_videos.loc[df_800_middle_20_videos["video"] == v_name, "folder"] = os.path.basename(path)
    
    print(f"videos in {path}: {videos_in_path}")

videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornEasy: 200
videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornDifficulty: 198
videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vPorn: 400


In [27]:
df_800_middle_20_videos

,video,label,frame_count,folder
0,vNonPorn1,0,20,vNonPornEasy
1,vNonPorn10,0,20,vNonPornDifficulty
2,vNonPorn100,0,20,vNonPornEasy
3,vNonPorn1006,0,20,vNonPornDifficulty
4,vNonPorn1007,0,20,vNonPornDifficulty
...,...,...,...,...
793,vPorn94,1,20,vPorn
794,vPorn95,1,20,vPorn
795,vPorn96,1,20,vPorn
796,vPorn97,1,20,vPorn


In [28]:
# videos from which it was not possible to extract 20 frames
df_800_middle_20_videos[df_800_middle_20_videos["frame_count"] < 20]

,video,label,frame_count,folder


In [29]:
# NOTE: I accidentally removed one video from its original location

print("number of videos:", len(df_800_middle_20_videos))
print("number of non-porn frames:", len(df_800_middle_20[df_800_middle_20["label"] == 0]))
print("number of porn frames:", len(df_800_middle_20[df_800_middle_20["label"] == 1]))

number of videos: 798
number of non-porn frames: 7960
number of porn frames: 8000


### Pornography-2k

#### middle-20

In [44]:
# check if data file is available
data_loc_2k_middle_20 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-2k-db/data-processed/middle-20"
os.path.isfile(f"{data_loc_2k_middle_20}/data.csv")

True

In [45]:
df_2k_middle_20 = pd.read_csv(f"{data_loc_2k_middle_20}/data.csv")
df_2k_middle_20["video"] = [frame_name.split("#")[0] for frame_name in df_2k_middle_20["frame"]]
df_2k_middle_20

,frame,label,video
0,vPorn000842#0.jpg,1,vPorn000842
1,vPorn000842#1.jpg,1,vPorn000842
2,vPorn000842#2.jpg,1,vPorn000842
3,vPorn000842#3.jpg,1,vPorn000842
4,vPorn000842#4.jpg,1,vPorn000842
...,...,...,...
39995,vPorn000431#15.jpg,1,vPorn000431
39996,vPorn000431#16.jpg,1,vPorn000431
39997,vPorn000431#17.jpg,1,vPorn000431
39998,vPorn000431#18.jpg,1,vPorn000431


In [46]:
agg = {"video": "first", "label": "first", "frame": "size"}
df_2k_middle_20_videos = df_2k_middle_20[["video", "frame", "label"]]
df_2k_middle_20_videos = df_2k_middle_20_videos.groupby("video").aggregate(agg).reset_index(drop=True)
df_2k_middle_20_videos = df_2k_middle_20_videos.rename(columns={"frame": "frame_count"})
df_2k_middle_20_videos

,video,label,frame_count
0,vNonPorn000001,0,20
1,vNonPorn000002,0,20
2,vNonPorn000003,0,20
3,vNonPorn000004,0,20
4,vNonPorn000005,0,20
...,...,...,...
1995,vPorn000996,1,20
1996,vPorn000997,1,20
1997,vPorn000998,1,20
1998,vPorn000999,1,20


In [47]:
# videos from which it was not possible to extract 20 frames
df_2k_middle_20_videos[df_2k_middle_20_videos["frame_count"] < 20]

,video,label,frame_count


In [48]:
print("number of videos:", len(df_2k_middle_20_videos))
print("number of non-porn frames:", len(df_2k_middle_20[df_2k_middle_20["label"] == 0]))
print("number of porn frames:", len(df_2k_middle_20[df_2k_middle_20["label"] == 1]))

number of videos: 2000
number of non-porn frames: 20000
number of porn frames: 20000


#### even-20

In [49]:
# check if data file is available
data_loc_2k_even_20 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-2k-db/data-processed/even-20"
os.path.isfile(f"{data_loc_2k_even_20}/data.csv")

True

In [50]:
df_2k_even_20 = pd.read_csv(f"{data_loc_2k_even_20}/data.csv")
df_2k_even_20["video"] = [frame_name.split("#")[0] for frame_name in df_2k_even_20["frame"]]
df_2k_even_20

,frame,label,video
0,vPorn000842#0.jpg,1,vPorn000842
1,vPorn000842#1.jpg,1,vPorn000842
2,vPorn000842#2.jpg,1,vPorn000842
3,vPorn000842#3.jpg,1,vPorn000842
4,vPorn000842#4.jpg,1,vPorn000842
...,...,...,...
39995,vPorn000431#15.jpg,1,vPorn000431
39996,vPorn000431#16.jpg,1,vPorn000431
39997,vPorn000431#17.jpg,1,vPorn000431
39998,vPorn000431#18.jpg,1,vPorn000431


In [51]:
agg = {"video": "first", "label": "first", "frame": "size"}
df_2k_even_20_videos = df_2k_even_20[["video", "frame", "label"]]
df_2k_even_20_videos = df_2k_even_20_videos.groupby("video").aggregate(agg).reset_index(drop=True)
df_2k_even_20_videos = df_2k_even_20_videos.rename(columns={"frame": "frame_count"})
df_2k_even_20_videos

,video,label,frame_count
0,vNonPorn000001,0,20
1,vNonPorn000002,0,20
2,vNonPorn000003,0,20
3,vNonPorn000004,0,20
4,vNonPorn000005,0,20
...,...,...,...
1995,vPorn000996,1,20
1996,vPorn000997,1,20
1997,vPorn000998,1,20
1998,vPorn000999,1,20


In [52]:
# videos from which it was not possible to extract 20 frames
df_2k_even_20_videos[df_2k_even_20_videos["frame_count"] < 20]

,video,label,frame_count


In [53]:
print("number of videos:", len(df_2k_even_20_videos))
print("number of non-porn frames:", len(df_2k_even_20[df_2k_even_20["label"] == 0]))
print("number of porn frames:", len(df_2k_even_20[df_2k_even_20["label"] == 1]))

number of videos: 2000
number of non-porn frames: 20000
number of porn frames: 20000


## Model training and testing

In [24]:
def separate_correct_and_incorrect(row):
    target = row["Target"]

    correct_frames = []
    incorrect_frames = []
    correct_confidences = []
    incorrect_confidences = []

    for frame, prediction, confidence in zip(row["Frame"], row["Prediction"], row["Confidence"]):
        if target == prediction:
            correct_frames.append(frame)
            correct_confidences.append(confidence)
        else:
            incorrect_frames.append(frame)
            incorrect_confidences.append(confidence)

    return pd.Series(
        [correct_frames, incorrect_frames, correct_confidences, incorrect_confidences],
        index=[
            "Correct_frames",
            "Incorrect_frames",
            "Correct_confidences",
            "Incorrect_confidences",
        ],
    )


def get_video_df_separated(df):
    df["Video"] = df["Frame"].str.split("#").str[0]

    # Group by video and aggregate columns
    video_df = (
        df.groupby("Video")
        .agg(
            {
                "Target": "first",
                "Prediction": lambda x: list(x),
                "Frame": lambda x: list(x),
                "Confidence": lambda x: list(x),
            }
        )
        .reset_index()
    )

    # Apply aggregation function
    video_df[
        [
            "Correct_frames",
            "Incorrect_frames",
            "Correct_confidences",
            "Incorrect_confidences",
        ]
    ] = video_df.apply(separate_correct_and_incorrect, axis=1)

    # Drop unnecessary columns
    video_df.drop(["Frame", "Prediction", "Confidence"], axis=1, inplace=True)

    return video_df


def true_positives(df):
    condition = lambda row: row["Target"] == 1 and len(row["Correct_frames"]) > len(row["Incorrect_frames"])
    return len(df[df.apply(condition, axis=1)])


def true_positives_relaxed(df):
    condition = lambda row: row["Target"] == 1 and len(row["Correct_frames"]) >= len(row["Incorrect_frames"])
    return len(df[df.apply(condition, axis=1)])


def true_negatives(df):
    condition = lambda row: row["Target"] == 0 and len(row["Correct_frames"]) > len(row["Incorrect_frames"])
    return len(df[df.apply(condition, axis=1)])


def true_negatives_relaxed(df):
    condition = lambda row: row["Target"] == 0 and len(row["Correct_frames"]) >= len(row["Incorrect_frames"])
    return len(df[df.apply(condition, axis=1)])


def false_positives(df):
    condition = lambda row: row["Target"] == 0 and len(row["Correct_frames"]) < len(row["Incorrect_frames"])
    return len(df[df.apply(condition, axis=1)])


def false_negatives(df):
    condition = lambda row: row["Target"] == 1 and len(row["Correct_frames"]) < len(row["Incorrect_frames"])
    return len(df[df.apply(condition, axis=1)])


def videos_correctly_classified(df, relaxed=False):
    return true_positives(df) + true_negatives(df) if not relaxed else true_positives_relaxed(df) + true_negatives_relaxed(df)


def videos_incorrectly_classified(df):
    return false_positives(df) + false_negatives(df)


def accuracy(df, relaxed=False):
    return videos_correctly_classified(df, relaxed) / len(df)


def precision(df, relaxed=False):
    tps = true_positives(df) if not relaxed else true_negatives_relaxed(df)
    return tps / (tps + false_positives(df))


def recall(df, relaxed=False):
    tps = true_positives(df) if not relaxed else true_negatives_relaxed(df)
    return tps / (tps + false_negatives(df))


def f1_score(df, relaxed=False):
    prec = precision(df, relaxed)
    rec = recall(df, relaxed)
    return 2 * (prec * rec) / (prec + rec)


def get_full_report(df):
    print(f"Total videos: {len(df)}\n")
    print(f"TPs: {true_positives(df)} | TNs: {true_negatives(df)} | FPs: {false_positives(df)} | FNs: {false_negatives(df)}\n")
    print(f"Correctly classified (TPs + TNs): {videos_correctly_classified(df)} (relaxed: {videos_correctly_classified(df, True)})")
    print(f"Incorrectly classified (FPs + FNs): {videos_incorrectly_classified(df)}\n")
    print("Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)")
    print(f"Accuracy: {accuracy(df):.5f} | Precision: {precision(df):.5f} | Recall: {recall(df):.5f} | F1-Score: {f1_score(df):.5f}\n")
    print("Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)")
    print(f"Accuracy: {accuracy(df, True):.5f} | Precision: {precision(df, True):.5f} | Recall: {recall(df, True):.5f} | F1-Score: {f1_score(df, True):.5f}\n")

### Pornography-800

#### VGG19

In [25]:
df = pd.read_csv("results/pornography-800/cnns/no-freeze/even-20/results/vgg19_freeze_False_epochs_100_batch_16_optim_sgd_aug_False_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 160

TPs: 78 | TNs: 75 | FPs: 5 | FNs: 1

Correctly classified (TPs + TNs): 153 (relaxed: 154)
Incorrectly classified (FPs + FNs): 6

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.95625 | Precision: 0.93976 | Recall: 0.98734 | F1-Score: 0.96296

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.96250 | Precision: 0.93750 | Recall: 0.98684 | F1-Score: 0.96154



#### VGG19 cross-dataset tested on Pornography-2k

In [26]:
df = pd.read_csv("results/cross-dataset-testing/train-800-test-2k/vgg19_freeze_False_epochs_100_batch_16_optim_sgd_aug_False_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 400

TPs: 169 | TNs: 191 | FPs: 6 | FNs: 26

Correctly classified (TPs + TNs): 360 (relaxed: 368)
Incorrectly classified (FPs + FNs): 32

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.90000 | Precision: 0.96571 | Recall: 0.86667 | F1-Score: 0.91351

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.92000 | Precision: 0.96985 | Recall: 0.88128 | F1-Score: 0.92344



#### ViT

In [27]:
df = pd.read_csv("results/pornography-800/transformers/no-freeze/even-20/results/vit_base_patch16_224_epochs_50_batch_16_optim_sgd_aug_False_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 160

TPs: 80 | TNs: 79 | FPs: 1 | FNs: 0

Correctly classified (TPs + TNs): 159 (relaxed: 159)
Incorrectly classified (FPs + FNs): 1

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.99375 | Precision: 0.98765 | Recall: 1.00000 | F1-Score: 0.99379

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.99375 | Precision: 0.98750 | Recall: 1.00000 | F1-Score: 0.99371



#### ViT cross-dataset tested on Pornography-2k

In [28]:
df = pd.read_csv("results/cross-dataset-testing/train-800-test-2k/vit_base_patch16_224_epochs_50_batch_16_optim_sgd_aug_False_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 400

TPs: 189 | TNs: 196 | FPs: 3 | FNs: 11

Correctly classified (TPs + TNs): 385 (relaxed: 386)
Incorrectly classified (FPs + FNs): 14

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.96250 | Precision: 0.98438 | Recall: 0.94500 | F1-Score: 0.96429

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.96500 | Precision: 0.98492 | Recall: 0.94686 | F1-Score: 0.96552



### Pornography-2k

#### VGG19

In [29]:
df = pd.read_csv("results/pornography-2k/cnns/data-aug/even-20/results/vgg19_freeze_False_epochs_50_batch_16_optim_sgd_aug_True_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 400

TPs: 191 | TNs: 191 | FPs: 6 | FNs: 9

Correctly classified (TPs + TNs): 382 (relaxed: 385)
Incorrectly classified (FPs + FNs): 15

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.95500 | Precision: 0.96954 | Recall: 0.95500 | F1-Score: 0.96222

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.96250 | Precision: 0.96985 | Recall: 0.95545 | F1-Score: 0.96259



#### VGG19 cross-dataset tested on Pornography-800

In [30]:
df = pd.read_csv("results/cross-dataset-testing/train-2k-test-800/vgg19_freeze_False_epochs_50_batch_16_optim_sgd_aug_True_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 160

TPs: 80 | TNs: 78 | FPs: 2 | FNs: 0

Correctly classified (TPs + TNs): 158 (relaxed: 158)
Incorrectly classified (FPs + FNs): 2

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.98750 | Precision: 0.97561 | Recall: 1.00000 | F1-Score: 0.98765

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.98750 | Precision: 0.97500 | Recall: 1.00000 | F1-Score: 0.98734



#### ViT 

In [31]:
df = pd.read_csv("results/pornography-2k/transformers/data-aug/even-20/results/vit_base_patch16_224_epochs_50_batch_16_optim_sgd_aug_True_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 400

TPs: 196 | TNs: 196 | FPs: 3 | FNs: 5

Correctly classified (TPs + TNs): 392 (relaxed: 392)
Incorrectly classified (FPs + FNs): 8

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.98000 | Precision: 0.98492 | Recall: 0.97512 | F1-Score: 0.98000

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.98000 | Precision: 0.98492 | Recall: 0.97512 | F1-Score: 0.98000



#### ViT cross-dataset tested on Pornography-800

In [32]:
df = pd.read_csv("results/cross-dataset-testing/train-2k-test-800/vit_base_patch16_224_epochs_50_batch_16_optim_sgd_aug_True_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 160

TPs: 80 | TNs: 79 | FPs: 1 | FNs: 0

Correctly classified (TPs + TNs): 159 (relaxed: 159)
Incorrectly classified (FPs + FNs): 1

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.99375 | Precision: 0.98765 | Recall: 1.00000 | F1-Score: 0.99379

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.99375 | Precision: 0.98750 | Recall: 1.00000 | F1-Score: 0.99371



### APD-2M

#### ResNet152 cross-dataset tested on Pornography-800

In [33]:
df = pd.read_csv("results/cross-dataset-testing/train-apd-test-800/resnet152_freeze_False_epochs_5_batch_256_optim_sgd_aug_True_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 160

TPs: 66 | TNs: 73 | FPs: 6 | FNs: 10

Correctly classified (TPs + TNs): 139 (relaxed: 144)
Incorrectly classified (FPs + FNs): 16

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.86875 | Precision: 0.91667 | Recall: 0.86842 | F1-Score: 0.89189

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.90000 | Precision: 0.92500 | Recall: 0.88095 | F1-Score: 0.90244



#### ResNet152 cross-dataset tested on Pornography-2k

In [34]:
df = pd.read_csv("results/cross-dataset-testing/train-apd-test-2k/resnet152_freeze_False_epochs_5_batch_256_optim_sgd_aug_True_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 400

TPs: 158 | TNs: 192 | FPs: 7 | FNs: 38

Correctly classified (TPs + TNs): 350 (relaxed: 355)
Incorrectly classified (FPs + FNs): 45

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.87500 | Precision: 0.95758 | Recall: 0.80612 | F1-Score: 0.87535

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.88750 | Precision: 0.96482 | Recall: 0.83478 | F1-Score: 0.89510



#### DenseNet201 cross-dataset tested on Pornography-800

In [35]:
df = pd.read_csv("results/cross-dataset-testing/train-apd-test-800/densenet201_freeze_False_epochs_5_batch_256_optim_sgd_aug_True_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 160

TPs: 59 | TNs: 70 | FPs: 9 | FNs: 20

Correctly classified (TPs + TNs): 129 (relaxed: 131)
Incorrectly classified (FPs + FNs): 29

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.80625 | Precision: 0.86765 | Recall: 0.74684 | F1-Score: 0.80272

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.81875 | Precision: 0.88750 | Recall: 0.78022 | F1-Score: 0.83041



#### DenseNet201 cross-dataset tested on Pornography-2k

In [36]:
df = pd.read_csv("results/cross-dataset-testing/train-apd-test-2k/densenet201_freeze_False_epochs_5_batch_256_optim_sgd_aug_True_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 400

TPs: 156 | TNs: 191 | FPs: 7 | FNs: 41

Correctly classified (TPs + TNs): 347 (relaxed: 352)
Incorrectly classified (FPs + FNs): 48

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.86750 | Precision: 0.95706 | Recall: 0.79188 | F1-Score: 0.86667

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.88000 | Precision: 0.96482 | Recall: 0.82403 | F1-Score: 0.88889



#### ViT (no aug) cross-dataset tested on Pornography-800

In [37]:
df = pd.read_csv("results/cross-dataset-testing/train-apd-test-800/vit_base_patch16_224_epochs_5_batch_256_optim_sgd_aug_False_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 160

TPs: 73 | TNs: 73 | FPs: 7 | FNs: 6

Correctly classified (TPs + TNs): 146 (relaxed: 147)
Incorrectly classified (FPs + FNs): 13

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.91250 | Precision: 0.91250 | Recall: 0.92405 | F1-Score: 0.91824

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.91875 | Precision: 0.91250 | Recall: 0.92405 | F1-Score: 0.91824



#### ViT (no aug) cross-dataset tested on Pornography-2k

In [38]:
df = pd.read_csv("results/cross-dataset-testing/train-apd-test-2k/vit_base_patch16_224_epochs_5_batch_256_optim_sgd_aug_False_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 400

TPs: 170 | TNs: 193 | FPs: 6 | FNs: 29

Correctly classified (TPs + TNs): 363 (relaxed: 365)
Incorrectly classified (FPs + FNs): 35

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.90750 | Precision: 0.96591 | Recall: 0.85427 | F1-Score: 0.90667

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.91250 | Precision: 0.96985 | Recall: 0.86937 | F1-Score: 0.91686



#### ViT (aug) cross-dataset tested on Pornography-800

In [39]:
df = pd.read_csv("results/cross-dataset-testing/train-apd-test-800/vit_base_patch16_224_epochs_5_batch_256_optim_sgd_aug_True_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 160

TPs: 70 | TNs: 75 | FPs: 4 | FNs: 7

Correctly classified (TPs + TNs): 145 (relaxed: 149)
Incorrectly classified (FPs + FNs): 11

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.90625 | Precision: 0.94595 | Recall: 0.90909 | F1-Score: 0.92715

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.93125 | Precision: 0.95000 | Recall: 0.91566 | F1-Score: 0.93252



#### ViT (aug) cross-dataset tested on Pornography-2k

In [40]:
df = pd.read_csv("results/cross-dataset-testing/train-apd-test-2k/vit_base_patch16_224_epochs_5_batch_256_optim_sgd_aug_True_split_10_20.csv")
df_videos = get_video_df_separated(df)
get_full_report(df_videos)

Total videos: 400

TPs: 163 | TNs: 195 | FPs: 4 | FNs: 33

Correctly classified (TPs + TNs): 358 (relaxed: 363)
Incorrectly classified (FPs + FNs): 37

Strict (i.e., a video is only correct if # correctly classified frames > # incorrectly classified frames)
Accuracy: 0.89500 | Precision: 0.97605 | Recall: 0.83163 | F1-Score: 0.89807

Relaxed (i.e., a video is correct if # correctly classified frames >= # incorrectly classified frames)
Accuracy: 0.90750 | Precision: 0.97990 | Recall: 0.85526 | F1-Score: 0.91335

